# Enzyme Modules

An "enzyme module" is defined as a mechanistic description of a reaction consisting of mass action rate laws for all known reaction steps <cite data-cite="DZK+16">(Du et al., 2016)</cite>. In **MASSpy**, enzyme modules are represented by the `EnzymeModule` object.

To demonstrate how an `EnzymeModule` object aids in constructing enzyme modules, an `EnzymeModule` of Hexokinase$^{1,2}$ will be constructed.

## Constructing Enzyme Modules

In order to construct the `EnzymeModule` of Hexokinase, the following information is necessary:

1. The enzyme is a monomer.
2. The enzyme binding of substrates follows a random sequential mechanism
3. The enzyme experiences product inhibtion, and is competitively inhibited by 23DPG when complexed with D-glucose.

Total HEX1 Concentration$^3$: \\(\text{[HEX1]}_{total} = 24 nM = 0.000024 mM\\) .

In [ ]:
from operator import attrgetter

from mass import MassMetabolite
from mass.enzyme_modules import EnzymeModule
from mass.test import create_test_model

# Load the glycolysis and hemoglobin models, then merge them
glycolysis = create_test_model("Glycolysis")
hemoglobin = create_test_model("Hemoglobin")
glyc_hb = glycolysis.merge(hemoglobin, inplace=False)

The `EnzymeModule` is a subclass of the `MassModel`, meaning that it inherits the methods and many of the behaviors of the `MassModel` object. 
Just like a `MassModel`, an `EnzymeModule` object requires a unique identifier to be created. Optionally, the `name` and `subsystem` attributes of the enzyme during initialization.

In [ ]:
HEX1 = EnzymeModule("HEX1", name="Hexokinase (D-glucose:ATP)",
                    subsystem="Glycolysis")

### Defining the enzyme ligands

The ligands that interact with the enzyme, such as the substrates, activators, inhibitors, are created as `MassMetabolite` objects and added to the model.

In [ ]:
glc__D_c = MassMetabolite(
    "glc__D_c",
    name="D-Glucose",
    formula="C6H12O6",
    charge=0,
    compartment="c")
g6p_c = MassMetabolite(
    "g6p_c",
    name="D-Glucose 6-phosphate",
    formula="C6H11O9P",
    charge=-2,
    compartment="c")
atp_c = MassMetabolite(
    "atp_c",
    name="ATP",
    formula="C10H12N5O13P3",
    charge=-4,
    compartment="c")
adp_c = MassMetabolite(
    "adp_c",
    name="ADP",
    formula="C10H12N5O10P2",
    charge=-3,
    compartment="c")
_23dpg_c = MassMetabolite(
    "_23dpg_c", 
    name="2,3-Disphospho-D-glycerate", 
    formula="C3H3O10P2",
    charge=-5,
    compartment="c")
h_c = MassMetabolite(
    "h_c",
    name="H+",
    formula="H",
    charge=1,
    compartment="c")

HEX1.add_metabolites([glc__D_c, g6p_c, atp_c, adp_c, _23dpg_c, h_c])

Once added to the `EnzymeModule`, the enzyme ligands can be accessed using the `enzyme_module_ligands` attribute.

In [ ]:
HEX1.enzyme_module_ligands

In order to keep track of the roles played by various ligands in the module, the `enzyme_module_ligands_categorized` attribute can be set. The attribute takes a `dict` where keys are categories and values are the relevant `MassMetabolite` objects. Note that an object can be a part of multiple categories.

In [ ]:
HEX1.enzyme_module_ligands_categorized =  {
    "substrates": glc__D_c,
    "cofactors": atp_c,
    "inhibitors": _23dpg_c,
    "products": [adp_c, g6p_c, h_c]}
HEX1.enzyme_module_ligands_categorized

For each category, a `cobra.Group` is created containing the relevant objects. Once set, the attribute will return a `cobra.DictList` containing the categorized groups. The groups and their members can be printed:

In [ ]:
for group in HEX1.enzyme_module_ligands_categorized:
    print("{0}: {1}".format(
        group.id, str(sorted([m.id for m in group.members]))))

### Defining the enzyme module forms

After adding `MassMetabolite` objects for the ligands to the model, the various forms of the enzyme must be defined. These forms are represented by `EnzymeModuleForm` objects. 

#### The EnzymeModuleForm Object

The `EnzymeModuleForm` object inherits from the `MassMetabolite` and therefore can be treated like any other metabolite in the model. However, the `EnzymeModuleForm` object contains the additional `bound_catalytic` and `bound_effectors` attributes to assist in tracking the metabolites bound to the enzyme form. 

The `EnzymeModule.make_enzyme_module_form` method allows for the creation of an `EnzymeModuleForm` object while assigning categories for the  `EnzymeModuleForm` in the process. Using `make_enzyme_module_form` also adds the species to the module upon creation, accessible via the `EnzymeModule.enzyme_module_forms` attribute.

In [ ]:
hex1_c = HEX1.make_enzyme_module_form(
    "hex1_c",
    name="automatic",
    categories="Free",
    compartment="c")

hex1_A_c = HEX1.make_enzyme_module_form(
    "hex1_A_c",  # A stands complexted with ATP
    name="automatic",
    categories="Free",
    bound_catalytic={atp_c: 1},
    compartment="c")

hex1_G_c = HEX1.make_enzyme_module_form(
    "hex1_G_c",  # G stands for complexed with Glucose
    name="automatic",
    categories="Free",
    bound_catalytic={glc__D_c: 1},
    compartment="c")

hex1_AG_c = HEX1.make_enzyme_module_form(
    "hex1_AG_c",
    name="automatic",
    categories="Free",
    bound_catalytic={glc__D_c: 1, atp_c: 1},
    compartment="c")

hex1_G_CI_c = HEX1.make_enzyme_module_form(
    "hex1_G_CI_c",  # CI stands for competitive inhibition
    name="automatic",
    categories="Inhibited",
    bound_catalytic={glc__D_c: 1, _23dpg_c: 1},
    compartment="c")

hex1_A_PI_c = HEX1.make_enzyme_module_form(
    "hex1_A_PI_c",  # PI stands for competitive inhibition
    name="automatic",
    categories="Inhibited",
    bound_catalytic={adp_c: 1},
    compartment="c")

hex1_G_PI_c = HEX1.make_enzyme_module_form(
    "hex1_G_PI_c",  # PI stands for competitive inhibition
    name="automatic",
    categories="Inhibited",
    bound_catalytic={g6p_c: 1},
    compartment="c")

HEX1.enzyme_module_forms

The `bound_catalytic` attribute represents the ligands bound to active or catalytic sites of enzyme, while the `bound_effectors` attribute is represents ligands bound to other sites, such as those for allosteric regulators. Note that these two attributes behave identically.

In [ ]:
# Print automatically generated names
for enzyme_form in HEX1.enzyme_module_forms:
    print("Bound to catalytic site of {0}:\n{1}\n".format(
        enzyme_form.id, enzyme_form.bound_catalytic))

Setting the `bound_catalytic` and `bound_effectors` attributes on creation allow the `formula` and `charge` attributes of the various forms to be set such that mass and charge balancing is maintained the model. Note that the enzyme is represented as a moiety and the chemical formula represents the moieties bound to the enzyme.

In [ ]:
# Get the elemental matrix for the enzyme
df = HEX1.get_elemental_matrix(array_type="DataFrame")
# Use iloc to only look at EnzymeModuleForms
df.iloc[:, 6:]

Setting the `name` argument as "automatic" in the `EnzymeModule.make_enzyme_module_form` method will cause a name for the `EnzymeModuleForm` to be generated based on the metabolites in the `bound_catalytic` and `bound_effectors` attributes.

In [ ]:
# Print automatically generated names
for enzyme_form in HEX1.enzyme_module_forms:
    print(enzyme_form.name)

The `categories` argument allows for `EnzymeModuleForm` objects to be placed into `cobra.Group` objects representing those categories. As with the ligands, the categorized enzyme module forms can be returned in a `DictList` of `Group` objects using the `enzyme_module_forms_categorized` attribute.

In [ ]:
for group in HEX1.enzyme_module_forms_categorized:
    print("{0}: {1}".format(
        group.id, str(sorted([m.id for m in group.members]))))

Alternatively, the `enzyme_module_forms_categorized` attribute can be set using a `dict`:

In [ ]:
HEX1.enzyme_module_forms_categorized =  {
    "competitively_inhibited": hex1_G_CI_c}

for group in HEX1.enzyme_module_forms_categorized:
    print("{0}: {1}".format(
        group.id, str(sorted([m.id for m in group.members]))))

### Defining enzyme module reactions

With the ligands and various enzyme forms defined, the next step is to define all of the reaction steps that represent the catalytic mechanism and regulation of the enzyme module. These reactions are represented as `EnzymeModuleReaction` objects. 

#### The EnzymeModuleReaction Object

The `EnzymeModuleReaction` object inherits from the `MassReaction` and therefore can be treated like any other reaction in the model. Like the `make_enzyme_module_form` method, the `make_enzyme_module_reaction` method allows for the creation of an `EnzymeModuleReaction` object while assigning categories for the `EnzymeModuleReaction` in the process.

Species that exist in the model can also be added to the reaction by passing a dictionary of metabolites and their stoichiometric coefficients to the `metabolites_to_add` argument.

In [ ]:
HEX1_1 = HEX1.make_enzyme_module_reaction(
    "HEX1_1",
    name="Automatic",
    subsystem="Glycolysis",
    reversible=True,
    categories="product_inhibition",
    metabolites_to_add={
        "hex1_c": -1,
        "adp_c": -1,
        "hex1_A_PI_c": 1})

HEX1_2 = HEX1.make_enzyme_module_reaction(
    "HEX1_2",
    name="Automatic",
    subsystem="Glycolysis",
    reversible=True,
    categories="product_inhibition",
    metabolites_to_add={
        "hex1_c": -1,
        "g6p_c": -1,
        "hex1_G_PI_c": 1})

HEX1_3 = HEX1.make_enzyme_module_reaction(
    "HEX1_3",
    name="Automatic",
    subsystem="Glycolysis",
    reversible=True,
    categories="glc__D_c_binding",
    metabolites_to_add={
        "hex1_c": -1,
        "glc__D_c": -1,
        "hex1_G_c": 1})

HEX1_4 = HEX1.make_enzyme_module_reaction(
    "HEX1_4",
    name="Automatic",
    subsystem="Glycolysis",
    reversible=True,
    categories="atp_c_binding",
    metabolites_to_add={
        "hex1_c": -1,
        "atp_c": -1,
        "hex1_A_c": 1})

HEX1_5 = HEX1.make_enzyme_module_reaction(
    "HEX1_5",
    name="Automatic",
    subsystem="Glycolysis",
    reversible=True,
    categories="competitive_inhibition",
    metabolites_to_add={
        "hex1_G_c": -1,
        "_23dpg_c": -1,
        "hex1_G_CI_c": 1})

HEX1_6 = HEX1.make_enzyme_module_reaction(
    "HEX1_6",
    name="Automatic",
    subsystem="Glycolysis",
    reversible=True,
    categories="atp_c_binding",
    metabolites_to_add={
        "hex1_G_c": -1,
        "atp_c": -1,
        "hex1_AG_c": 1})

HEX1_7 = HEX1.make_enzyme_module_reaction(
    "HEX1_7",
    name="Automatic",
    subsystem="Glycolysis",
    reversible=True,
    categories="glc__D_c_binding",
    metabolites_to_add={
        "hex1_A_c": -1,
        "glc__D_c": -1,
        "hex1_AG_c": 1})

HEX1_8 = HEX1.make_enzyme_module_reaction(
    "HEX1_8",
    name="Automatic",
    subsystem="Glycolysis",
    reversible=True,
    categories="catalyzation",
    metabolites_to_add={
        "hex1_AG_c": -1,
        "hex1_c": 1,
        "adp_c": 1,
        "g6p_c": 1,
        "h_c": 1})

for reaction in HEX1.enzyme_module_reactions:
    print(reaction)

The `categories` argument allows for `EnzymeModuleReactions` objects to be placed into `cobra.Group` objects representing those categories. As with the ligands and enzyme forms, a `DictList` of the relevant groups can be returned with the `enzyme_module_reactions_categorized` attribute.

In [ ]:
HEX1.enzyme_module_reactions_categorized

#### Unifying rate parameters

For this `EnzymeModule`, the reactions representing glucose binding to the enzyme and ATP binding to the enzyme have the same forward rate and equilibrium constants. Instead of defining the parameter values for each individual reaction, the `unify_rate_parameters` method can be used to create custom rate laws for the given reactions that all depend on the same rate parameters.

The `unify_rate_parameters` takes a list of reactions and an identifier to use for the unified parameter. The `enzyme_prefix` flag can be set to `True` to prefix the new parameter identifier with the identifier of the `EnzymeModule`, ensuring that any existing custom parameters are not overwritten.

In [ ]:
for ligand, pid in zip([glc__D_c, atp_c],["G", "A"]):
    # Get the group of reactions corresponding to the ligand
    category = "_".join((ligand.id, "binding"))
    group = HEX1.enzyme_module_reactions_categorized.get_by_id(category)
    
    # Unify the parameters
    HEX1.unify_rate_parameters(
        group.members, new_parameter_id=pid, enzyme_prefix=True)

    # Print the new reaction rates
    print("\n" + category + "\n" + "-" * len(category))
    for reaction in sorted(group.members, key=attrgetter("id")):
        print(reaction.id + ": " + str(reaction.rate))

## Determining steady state concentrations and rate constants

The next step is to solve for the steady state concentrations for the various forms of the enzyme symbolically using **sympy**. Because the numerical values for the dissociation constants have been defined, these equations will be solved in terms of the rate constants. The rate constants can be approximated using the total enzyme concentration as a constraint and substituted back into the equations to calculate the numerical values of the steady state concentrations.

In [ ]:
from sympy import Eq, Symbol, lambdify, simplify, solveset

from mass import strip_time
from mass.util.matrix import matrix_rank

### Solving steady state concentrations symbolically

To get the symbolic solutions for the individual enzyme forms, the ODEs are first collected in a `dict` where keys are the enzyme forms and values are their ODEs without time dependency:

In [ ]:
ode_dict = {
    enzyme_form.id: Eq(strip_time(enzyme_form.ode))
    for enzyme_form in HEX1.enzyme_module_forms}
# Matrix rank of enzyme stoichiometric matrix without substrates
rank = matrix_rank(HEX1.S[6:])
print("Rank Deficiency: {0}".format(len(ode_dict) - rank))

Because the stoichiometric matrix (without ligands) has a rank deficiency of 1, there will be a dependent variable unless another equation is added. Therefore the completely free enzyme form is treated as the dependent variable and all of the enzyme forms are solved in terms of the free enzyme form:

In [ ]:
enzyme_solutions = {}
for enzyme_form in HEX1.enzyme_module_forms:
    # Skip dependent variable
    if enzyme_form.id == "hex1_c":
        continue
    # Get the ODE for the enzyme form from the ODE dict
    equation = ode_dict[enzyme_form.id]
    # Solve the equation for the enzyme form, substituting 
    # previously found enzyme form solutions into the equation
    solution = solveset(equation.subs(enzyme_solutions),
                        enzyme_form.id)
    # Store the solution
    enzyme_solutions[enzyme_form.id] = list(solution)[0]
    # Substitute the new solution into existing solutions
    enzyme_solutions.update({
        enzyme_form: sol.subs(enzyme_solutions) 
        for enzyme_form, sol in enzyme_solutions.items()})
    
args = set()
for solution in enzyme_solutions.values():
    args.update(solution.atoms(Symbol))

#### Defining the Rate Equation
In order to make up for the rank deficiency, an additional equation is needed. Typically, the rate of the enzyme is the summation of the rates for the catalyzation reactions of the enzyme. The `make_enzyme_rate_equation` method can be used to create the rate equation from a list of reactions. If `use_rates` is set as True, the rate laws will be added together. If the `update_enzyme` flag is set as `True`, the rate equation will be stored as a sympy expression in the `enzyme_rate_equation` attribute.

In [ ]:
# Get the catalyzation reactions
catalyzation_group = HEX1.enzyme_module_reactions_categorized.get_by_id(
    "catalyzation")

HEX1.make_enzyme_rate_equation(catalyzation_group.members,
                               use_rates=True,
                               update_enzyme=True)

print(HEX1.enzyme_rate_equation)

With the rate equation defined, the `enzyme_rate_error` method can be used to get the equation as the difference between the steady state flux and the rate equation.

In [ ]:
enzyme_rate_equation = strip_time(HEX1.enzyme_rate_error(use_values=False))
print(enzyme_rate_equation)

The enzyme form solutions are substituted into the rate equation and the equation is solved for the free enzyme form. The solutions are then updated, resulting in symbolic equations that do not depend on any enzyme form.

In [ ]:
# Solve for last unknown concentration symbolically
solution = solveset(enzyme_rate_equation.subs(enzyme_solutions),
                    "hex1_c")

# Update solution dictionary with the new solution
enzyme_solutions["hex1_c"] = list(solution)[0]

# Update solutions with free variable solutions
enzyme_solutions = {
    enzyme_form: simplify(solution.subs(enzyme_solutions))
    for enzyme_form, solution in enzyme_solutions.items()}

args = set()
for solution in enzyme_solutions.values():
    args.update(solution.atoms(Symbol))
print(args)

Numerical values for known quantities can be substituted into the equations. For this `EnzymeModule` of Hexokinase, the following dissociation constants are used:

$$\begin{align}
K_{d,\ \text{GLC-D}} &= 0.038\ \text{mM} \\
K_{d,\ \text{ATP}} &= 2.06\ \text{mM} \\
K_{i,\ \text{23DPG}} &= 5.5\ \text{mM} \\
K_{i,\ \text{ADP}} &= 1\ \text{mM} \\
K_{i,\ \text{G6P}} &= 66.67\ \text{mM} \\
\end{align}$$

A value of $K_{\text{HEX1}}= 313.12$ will be used for the catalyzation step. Note that the inverse of the dissociation constant is used for reactions that form complexes. 

In [ ]:
numerical_values = {
    "Keq_HEX1_1": 1,
    "Keq_HEX1_2": 1 / 66.67,
    "Keq_HEX1_G": 1 / 0.038,  
    "Keq_HEX1_A": 1 / 2.06,
    "Keq_HEX1_5": 1 / 5.5,
    "Keq_HEX1_8": 313.12}
# Update the model with the parameters
HEX1.update_parameters(numerical_values)

The ligand concentrations and the rate for the enzyme are extracted from the glycolysis model merged with the hemoglobin model.

In [ ]:
# Get steady state flux for EnzymeModule
HEX1.enzyme_rate = glyc_hb.reactions.get_by_id("HEX1").steady_state_flux
numerical_values[HEX1.enzyme_flux_symbol_str] = HEX1.enzyme_rate

# Get the ligand concentrations
for met in HEX1.enzyme_module_ligands:
    concentration = glyc_hb.metabolites.get_by_id(met.id).initial_condition
    # Set the ligand initial condition and add to numercal values dictionary
    met.initial_condition = concentration
    numerical_values[met.id] = concentration

The numerical values are substituted into the symbolic equations, resulting in the steady state concentrations being dependent only on the rate constants.

In [ ]:
enzyme_solutions = {
    enzyme_form: simplify(sol.subs(numerical_values))
    for enzyme_form, sol in enzyme_solutions.items()}

args = set()
for solution in enzyme_solutions.values():
    args.update(solution.atoms(Symbol))
print(args)

### Approximating Rate Constants

In order to determine the set of rate constants for this enzyme module, the absolute error between the total hexokinase concentration value found in literature, and the computed hexokinase concentration based on the choice of rate constants. Here, the `minimize` function of the **scipy** package will bsed utilized to approximate the rate constants. 

In [ ]:
from scipy.optimize import minimize

The objective function for the minimization will first be made symbolically. The `enzyme_total_symbol_str` property can be used to represent the total enzyme concentration, while the `enzyme_concentration_total_equation` property will create the sum of all enzyme forms as a symbolic expression.

In [ ]:
enzyme_total_error = abs(
    Symbol(HEX1.enzyme_total_symbol_str)
    - strip_time(HEX1.enzyme_concentration_total_equation))
print(enzyme_total_error)

The `enzyme_concentration_total` attribute can store the total amount of enzyme in the model and substiuted into the expression. The total HEX1 concentration is $24 * 10^{-6} \text{mM}$.

In [ ]:
HEX1.enzyme_concentration_total = 24e-6
enzyme_total_error = enzyme_total_error.subs({
    HEX1.enzyme_total_symbol_str: HEX1.enzyme_concentration_total})
print(enzyme_total_error)

Finally, the symbolic equations for the enzyme forms can be substituted into the equation, resulting in a symbolic expression representing the objective function for minimization, where the only unknowns variables are the rate constants. The `lambdify` function of the **sympy** package can convert the symbolic objective into a lambda function that can be used with the `minimize` function of **scipy**.

In [ ]:
enzyme_total_error = simplify(enzyme_total_error.subs(enzyme_solutions))

# Sort the arguments to ensure input format remains consistent
args = sorted(list(map(str, args)))
# Use lambdify to make objective function as a lambda function
obj_fun = lambda x: lambdify(args, enzyme_total_error)(*x)

The `minimize` function can be used to approximate the rate constants. Note that these problems are typically nonlinear and therefore require a nonlinear optimization routine to find a feasible solution.

In [ ]:
# Minimize the objective function, initial guess based on publication values
initial_guess = [1e8, 9376585, 52001]
variable_bounds = ((0, 1e9), (0, 1e9), (0, 1e9))
solution = minimize(obj_fun, x0=initial_guess,
                    method="trust-constr",
                    bounds=variable_bounds)
# Map solution array to variables
rate_constants = dict(zip(args, solution.x))
print(rate_constants)

Because the rate constants associated with the inhibition of the enzyme forms were not necessary for the computing the steady state concentrations, a rapid binding assumption is made for these reactions and a large number can be set for the rate constants. The parameters are set for the reactions using the `update_parameters` method.

In [ ]:
rate_constants["kf_HEX1_1"] = 1e6
rate_constants["kf_HEX1_2"] = 1e6
rate_constants["kf_HEX1_5"] = 1e6
HEX1.update_parameters(rate_constants)

### Calculating numerical values for steady state concentrations

Once the rate constants have been estimated, they can be substituted back  the symbolic steady state concentrations equations in order to obtain their corresponding numerical values

In [ ]:
for enzyme_form, solution in enzyme_solutions.items():
    # Get the enzyme form object, determine the steady state concentration
    enzyme_form = HEX1.enzyme_module_forms.get_by_id(enzyme_form)
    enzyme_form.initial_condition = float(solution.subs(rate_constants))
    print("{0}: {1:e}".format(enzyme_form.id,
                              enzyme_form.initial_condition))

#### Error values

As a quality assurance check, the `enzyme_concentration_total_error` method can be used to get the error between the `enzyme_concentration_total` attribute and the sum of the enzyme form concentrations. A positive value indicates the `enzyme_concentration_total` attribute is greater than the sum of the individual enzyme form concentrations that were computed.

In [ ]:
print("Total Enzyme Concentration Error: {0}".format(
    HEX1.enzyme_concentration_total_error(use_values=True)))

Similarly the error between the `enzyme_rate` attribute and the computed value from the `enzyme_rate_equation` can be checked as well using the `enzyme_rate_error` method, where A positive value indicates the `enzyme_rate` attribute is greater than the value computed using the rate equation.

In [ ]:
print("Enzyme Rate Error: {0}".format(
    HEX1.enzyme_rate_error(use_values=True)))

## Adding EnzymeModules to Models

With the `EnzymeModule` has been built, it can be integrated into a larger network and simulated. To add an `EnzymeModule` to an existing `MassModel`, the `merge` method can be used. After merging, the `remove_reactions` method is used to remove the reaction that was just replaced with the enzyme module. Note that the `EnzymeModule` should always be merged into the `MassModel` as demonstrated below:

In [ ]:
glyc_hb_HEX1 = glyc_hb.merge(HEX1, inplace=False)
glyc_hb_HEX1.remove_reactions([
    glyc_hb_HEX1.reactions.get_by_id("HEX1")])

Everything in the `EnzymeModule` is transfered into the `MassModel` upon merging, including all enzyme forms, reactions steps, initial conditions, rate parameters, and category groups.

In [ ]:
glyc_hb_HEX1

### The EnzymeModuleDict object

During the merge process, an `EnzymeModuleDict` is created from the `EnzymeModule` and added to the `MassModel.enzyme_modules` attribute.

In [ ]:
print(glyc_hb_HEX1.enzyme_modules)
HEX1_dict = glyc_hb_HEX1.enzyme_modules.get_by_id("HEX1")
HEX1_dict

The `EnzymeModuleDict` inherits from an `OrderedDict`, thereby inheriting the dictionary methods:

In [ ]:
print("\n".join(HEX1_dict.keys()))

The `EnzymeModuleDict` stores several of the enzyme-specific attributes so that they can still be accessed after the enzyme module is integrated into a larger network. The keys of the `EnzymeModuleDict`  can also be treated as attribute accessors:

In [ ]:
print("Enzyme Rate:\n{0} = {1}".format(
    HEX1_dict["enzyme_rate"],       # Returned using dict key
    HEX1_dict.enzyme_rate_equation  # Returned using attribute accessor
))

### Steady State Validation
The last step is to ensure that a steady state can be reached with the completed enzyme module within a larger network context.

In [ ]:
import matplotlib.pyplot as plt

from mass import Simulation
from mass.visualization import plot_time_profile

Here, the model is simulated and the enzyme's ability to reach steady state is graphically verified: 

In [ ]:
# Setup simulation object
sim = Simulation(glyc_hb_HEX1, verbose=True)
# Simulate from 0 to 1000 with 10001 points in the output
conc_sol, flux_sol = sim.simulate(
    glyc_hb_HEX1, time=(0, 1e3, 1e4 + 1))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
plot_time_profile(
    conc_sol, observable=HEX1_dict.enzyme_module_forms, ax=ax,
    legend="right outside", plot_function="loglog",
    xlabel="Time [hr]", ylabel="Concentration [mM]",
    title="TIme profile of Concentrations for Enzyme Forms");

The plot shows that the enzyme can reach a steady state when integrated into a larger network, meaning the enzyme module representing hexokinase in for this system is complete!

$^1$ Procedure outlined <cite data-cite="DZK+16">(Du et al., 2016)</cite>

$^2$ Hexokinase enzyme module based on <cite data-cite="YAHP18">(Yurkovich et al., 2018)</cite> and <cite data-cite="DZK+16">(Du et al., 2016)</cite>.

$^3$ Total hexokinase concentration from <cite data-cite="MK99">(Mulquiney and Kuchel, 1999).</cite>